<a href="https://colab.research.google.com/github/preenet/ds_talk_nlp/blob/main/DS_talk_Intro_to_NLP_Workshop_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pree Thiengburanathum
# pree.t@cmu.ac.th
# workshop part 1
# We use wisesight sentiment corpus, can be downloaded at 
https://github.com/PyThaiNLP/wisesight-sentiment/ or 

https://www.kaggle.com/c/wisesight-sentiment/data  
Perform text classification on Thai social media posts 


train.txt - the training data. Each line is a piece of text.  
train_label.txt - the label of the training data. Each line is the label   corresponding to the same line in train.txt.  
test.txt - the test data. Each line is a piece of text.  
test_majority.csv - a sample submission file using the majority class in the correct format.  
The Id in test_majority.csv is the line number in test.txt 

no feature engineering in this notebook, teaching proposed only!  



In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
import re
import pandas as pd
from matplotlib import pyplot as plt

!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
!pip install emoji
from pythainlp import word_tokenize


     / 11.7 MB 133 kB/s
     |████████████████████████████████| 743 kB 26.6 MB/s 
  Created wheel for pythainlp: filename=pythainlp-3.0.0.dev0-py3-none-any.whl size=11502796 sha256=66373665daba19afc31323a69d5374b737a4087ef3e950bf020fd0b5fdbd24b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-0r8bl4yh/wheels/87/69/e8/234fe11c5f2ec389a780229b2a1619e17b8ad38cd45eaf20f7
Successfully built pythainlp
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.2 which is incompatible.
     |████████████████████████████████| 168 kB 36.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.0-p

In [6]:
import emoji
from google.colab import drive
drive.mount('/content/gdrive')
data_path = '/content/gdrive/My Drive/Colab Notebooks/WiseSight Sentiment/kaggle-competition/'

Mounted at /content/gdrive


In [7]:
!ls '/content/gdrive/My Drive/Colab Notebooks/WiseSight Sentiment/kaggle-competition'

competition.ipynb  test_majority.csv  text_generation.ipynb  train.txt
README.md	   test_solution.csv  train_label.txt
test_label.txt	   test.txt	      train_model.py


In [8]:
# we open the given training set, namely train.txt and the target feature, namely train_label.txt
# then, we save as csv to the current directory.
features = []
targets = []

with open(data_path + "train.txt") as f:
    for line in f:
        features.append(line.strip())

with open(data_path+ "train_label.txt") as f:
    for line in f:
        targets .append(line.strip())

df = pd.DataFrame({ "targets": targets, "features": features })

df.to_csv("train.csv", index=False)
df.shape

(24063, 2)

In [9]:
# we do the sampe for testing
features = []
targets = []

with open(data_path + "test.txt") as f:
    for line in f:
        features.append(line.strip())

with open(data_path+ "test_label.txt") as f:
    for line in f:
        targets .append(line.strip())

df_test = pd.DataFrame({ "targets": targets, "features": features })

df_test.to_csv("test.csv", index=False)
df_test.shape

(2674, 2)

In [10]:
df.head(3)

,targets,features
0,neu,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ
1,neu,คะ
2,neg,อิเหี้ยออมทำกูอยากกินเอ็มเค


In [11]:
# check for any missing vlue
df.isnull().sum()

targets     0
features    0
dtype: int64

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic = LogisticRegression(C=2., penalty='l2', solver='liblinear', dual=False, multi_class='ovr')
# demonstrate w/o pipelining
coun_vect = CountVectorizer()
count_matrix = coun_vect.fit_transform( df['features'] )

tfidf_trans = TfidfTransformer()
vectors = tfidf_trans.fit_transform(count_matrix)

# demonstrate using pipelining
#logis = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("tfidf", TfidfTransformer()), ("logistic",  logistic)])

In [70]:
score = cross_val_score( logis, df['features'].to_numpy(), df['targets'].to_numpy(), cv = 5).mean()

In [71]:
print(score)

0.6041636334401201


In [72]:
#lets pre-processing the features :(
# took it from https://github.com/PyThaiNLP/wisesight-sentiment/blob/master/kaggle-competition/competition.ipynb
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def process_text(text):
    #pre rules
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)
    
    #tokenize
    res = [word for word in word_tokenize(res) if word and not re.search(pattern=r"\s+", string=word)]
    
    #post rules
    res = ungroup_emoji(res)
    
    return res

In [73]:
# let's do it properly :)

pos_df = df[df.targets == "pos"]
neu_df = df[df.targets == "neu"]
neg_df = df[df.targets== "neg"]

In [74]:
sentiment_df = pd.concat([neg_df, pos_df, neu_df])
sentiment_df = sentiment_df.reset_index(drop=True)
sentiment_df.groupby("targets").features.describe()

,count,unique,top,freq
targets,,,,
neg,6140,6129,#ERROR!,12
neu,13105,13087,#ERROR!,19
pos,4300,4300,แปะไว้ก่อนนะ,1


In [75]:
sentiment_df['processed'] = sentiment_df.features.map(lambda x: '|'.join(process_text(x)))

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sentiment_df['processed'], sentiment_df.targets, test_size=0.20, random_state=42)

In [77]:
score = cross_val_score( logis, sentiment_df['processed'].to_numpy(), sentiment_df.targets.to_numpy(), cv = 5).mean()

In [78]:
print(score)

0.6220004247186239


In [82]:
# test with unseen data
logis.predict(df_test)

NotFittedError: ignored